In [1]:
import cv2
import numpy as np
import os
import torch
import torchvision.transforms as transforms
from PIL import Image

from corner_detector_linear import CornerDetectorLinear
from ultralytics import YOLO

In [9]:
resnet_model = CornerDetectorLinear()
resnet_model.load_state_dict(torch.load('models/corner_detector_test2.pth'))
transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
            ])


In [5]:
yolo_model = YOLO('models/medium_best.pt')

In [6]:
def calculate_euclidean_error_ordered(original, predicted):
    error = 0
    for (x1, y1), (x2, y2) in zip(original, predicted):
        error += np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
    return error / len(original)


In [7]:
def calculate_euclidean_error_closest(original, predicted):
    error = 0
    for (x1, y1) in original:
        min_dist = float('inf')
        for (x2, y2) in predicted:
            dist = np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
            if dist < min_dist:
                min_dist = dist
        error += min_dist
    return error / len(original)


In [ ]:
test_images_folder = 'test_images'
images = [f for f in os.listdir(test_images_folder) if f.endswith('.jpg')]

In [16]:
resnet_errors_ordered = []
resnet_errors_closest = []
yolo_errors = []

for image in images:
    image_path = os.path.join(test_images_folder, image)
    label_path = image_path.replace('.jpg', '.txt')

    original_corners = []
    with open(label_path, 'r') as f:
        for line in f:
            x_str, y_str = line.strip().split(",")
            original_corners.append((float(x_str), float(y_str)))

    resnet_input_image = Image.open(image_path).convert('RGB')
    resnet_input_image = transform(resnet_input_image).unsqueeze(0)
    with torch.no_grad():
        resnet_output = resnet_model(resnet_input_image)
        resnet_corners = resnet_output.squeeze().numpy().reshape(-1, 2)
    resnet_errors_ordered.append(calculate_euclidean_error_ordered(original_corners, resnet_corners))
    resnet_errors_closest.append(calculate_euclidean_error_closest(original_corners, resnet_corners))

    yolo_input_image = cv2.imread(image_path)
    yolo_results = yolo_model(yolo_input_image, verbose=False)

    yolo_corners = []
    for result in yolo_results:
        for kpt in result.keypoints.xyn[0]:
            yolo_corners.append((float(kpt[0]), float(kpt[1])))
    yolo_errors.append(calculate_euclidean_error_closest(original_corners, yolo_corners))

print(f"ResNet Average Euclidean Error (Ordered): {np.mean(resnet_errors_ordered)}")
print(f"ResNet Average Euclidean Error (Closest): {np.mean(resnet_errors_closest)}")
print(f"YOLO Average Euclidean Error: {np.mean(yolo_errors)}")

ResNet Average Euclidean Error (Ordered): 0.5016036033630371
ResNet Average Euclidean Error (Closest): 0.2473767101764679
YOLO Average Euclidean Error: 0.0353339342221957
